# MolSim 2020: ML for Gas Adsorption

In this exercise we will build a model that can predict the CO$_2$ uptake of metal-organic frameworks (MOFs), which are crystalline materials consisting of inorganic metal nodes linked by organic linkers.

![MOF building principle](_static/mof_building_principle.png)

There are two main **learning goals** for this exercise: 
1. Understand the typical workflow for machine learning in materials science. We will cover exploratory data analysis (EDA) and supervised learning (KRR).
2. Learn about some Python packages that are useful for data analysis and visualization. If you are not already familiar with Python, this exercise can be a great opportunity to learn some basics.

At the end of the exercise, you will produce an interactive plot like the one below, comparing the predictions of your model against CO$_2$ computed with GCMC simulations.
The histograms show the distributions of the errors on the training set (left) and on the test set (right).

![Parity interactive](_static/result.gif)

This exercise requires a basic knowledge of Python, e.g. that you can write list comprehensions, and are able to read documentation of functions provided by Python packages.
You will be asked to provide some function arguments (indicated by `#fillme` comments).

If you struggle with this, click on the provided hints and/or partner up with someone more experienced. 
For those who already have experience with both Python and machine learning, there are also several optional exercises at the end.

You can execute all the following code cells by pressing SHIFT and ENTER and get informations about the functions by pressing TAB when you are between the parentheses (see the notes for more tips). Also the [sklearn documentation](https://scikit-learn.org/stable/user_guide.html) is a great source of reference with many explanations and examples.

In pandas dataframe (df) you can select columns using their name by running `df[columnname]`. 

## -1. Only if you run on Colab

If you use this notebook on Colab, please uncomment the lines below (remove the `#`) and execute the cell.

In [ ]:
#!pip install --upgrade pandas pandas-profiling sklearn holoviews bokeh plotly matplotlib
#!wget https://raw.githubusercontent.com/kjappelbaum/ml_molsim2020/master/descriptornames.py

## 0. Import packages we will need

In [ ]:
# basics 
import os 
import numpy as np 
import pprint as pp

# data
import pandas as pd 
import pandas_profiling

# machine learning 
# scaling of data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# train/test split
from sklearn.model_selection import train_test_split
# model selection 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# model
from sklearn.kernel_ridge import KernelRidge
# pipeline 
from sklearn.pipeline import Pipeline
# PCA
from sklearn.decomposition import PCA
# Dummy model
from sklearn.dummy import DummyClassifier, DummyRegressor
# Variance Threshold 
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
# metrics 
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             mean_absolute_error, mean_squared_error, max_error)
# feature names
from descriptornames import * 

# save/load models 
import joblib

# For the permutation importance implementation
from joblib import Parallel
from joblib import delayed
from sklearn.metrics import check_scoring
from sklearn.utils import Bunch
from sklearn.utils import check_random_state
from sklearn.utils import check_array

# plotting 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns

# for interactive plots, you can try to use holoviewes
import holoviews as hv
from holoviews import dim, opts
hv.extension('plotly', 'bokeh', 'matplotlib')

RANDOM_SEED = 4242424242
DATA_DIR = 'data'
DATA_FILE = os.path.join(DATA_DIR, 'data.csv')

np.random.seed(RANDOM_SEED)

def view_structure(name):
    import webbrowser
    webbrowser.open(f'https://discover.materialscloud.org/mofs/detail?name={name}', new=2)

 $\color{DarkBlue}{\textsf{Short question}}$
- We declared a global variable to fix the random seed (`RANDOM_SEED`). Why did we do this?  

## 1. Import the data

In [ ]:
df = pd.read_csv(DATA_FILE)

Let's take a look at the first few rows to see if everythings seems reasonable ...

In [ ]:
df.head()

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li>Use something like <code>pd.options.display.max_columns=10</code> to adjust how many columns are shown.</li>
</ul>
</details>

Let's also get some basic information ...

In [ ]:
df.info()

 $\color{DarkBlue}{\textsf{Short question}}$
- How many materials are in the dataset? 
- Which datatypes do we deal with?

Below, we define three global variables (hence upper case), which are the names of our feature and target columns. 

In [ ]:
TARGET = 'pure_uptake_CO2_298.00_1600000' 
TARGET_BINARY = 'target_binned'  # will be created later
FEATURES = (geometric_descriptors + summed_functionalgroup_descriptors 
            + summed_linker_descriptors + summed_metalcenter_descriptors)

As descriptors we will use geometric properties such as density, pore volume, etc. and [revised autocorrelation functions](https://pubs.acs.org/doi/abs/10.1021/acs.jpca.7b08750) (RACs) that have been optimized for describing inorganic compounds. 

Examples for pore geometry descriptors (in `geometric_descriptors`) include: $D_i$ (the size of the largest included sphere), $D_f$ (the largest free sphere), and $D_{if}$ (the largest included free sphere) along the pore $-$ three ways of characterizing pore size. 
![pore diameters](_static/spheres.png)

Also included are the surface area (SA) of the pore, and the probe-occupiable pore volume (POV).
More details on the description of pore geometries can be found in [Ongari et al.](https://pubs.acs.org/doi/abs/10.1021/acs.langmuir.7b01682)

RACs (in the lists starting with `summed_...`) operate on the structure graph and encode information about the metal center, linkers and the functional groups as differences or products of heuristics that are relevant for inorganic chemistry, such as electronegativity ($\chi$), connectivity ($T$), identity ($I$), covalent radii ($S$), and nuclear charge ($Z$).

![RACs scheme from the lecture](_static/racs.png)

The number in the descriptornames shows the coordination shell that was considered in the calculation of the RACs.

The target we use for this application is the high-pressure CO$_2$ uptake. This is the amount of CO$_2$ (mmol) the MOF can load per gram.

## 2. Split the data

Next, we split our data into a training set and a test set.

In order to prevent *any* information of the test set from leaking into our model, we split *before* starting to analyze or transform our data. For more details on why this matters, see [chapter 7.10.2 of Elements of Statistical Learning](https://web.stanford.edu/~hastie/ElemStatLearn//printings/ESLII_print10.pdf).

### 2.1. Split with stratification

Ideally, our model should perform well both on high-performing materials and on low-performing materials.
If there is a large imbalance between the number of high-performing and low-performing materials in the data set (e.g. few good materials), we might end up with almost no good materials in the test data set.

[Stratification](https://en.wikipedia.org/wiki/Stratified_sampling) ensures that the class distributions (ratio of good to bad materials) are the same in the training and test set.

For stratification to work, we to define what makes a good or a bad material. Following [Boyd et al.](https://www.nature.com/articles/s41586-019-1798-7), we will use 2 mmol CO$_2$ / g as the threshold for the uptake, thus binarizing our continuous target variable.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
 - add a column 'target_binary' that encodes whether a material is low performing (`0`) or high perfoming (`1`).

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> you can use <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html'>pd.cut</a>, 
    <a href='https://stackoverflow.com/questions/4406389/if-else-in-a-list-comprehension'>list comprehension</a>, the <a href='https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Binarizer.html#sklearn.preprocessing.Binarizer'> binarizer in sklearn </a>...) </li>
    <li> a list comprehension example: <code> [1 if value > THRESHOLD else 0 for value in df[TARGET]] </code> </li>
</ul>
</details>

In [ ]:
THRESHOLD = 2 # in units of mmol CO2/g
df['target_binned'] = # add your code

Now, we can perform the actual split into training and test set.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- select reasonable values for XX and XY and then perform the test/train splits. What do you consider when making this decision? 

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li>The size can either be integers or---easier---decimals like 0.1</li>
    <li>When you perform the split into training and test set you need to trade-off bias (pessimistic bias due to little training data) and variance (due to little test data) </li>
    <li>A typical split cloud be 70/30 </li>
</ul>
</details>

In [ ]:
df_train_stratified, df_test_stratified = train_test_split(df, train_size=XX, 
                                                                test_size=XY, 
                                                                random_state=RANDOM_SEED, 
                                                                stratify=df['target_binned']) 

## 3. Exploratory data analysis (EDA) 

After we have put the test set aside, we can give the training set a closer look.

### 3.1. Get some description of the data

Use the [`pandas_profiling`](https://github.com/pandas-profiling/pandas-profiling) package to get an overview of the data.

$\color{DarkBlue}{\textsf{Short Exercise}}$
- Are there skewed distributions? 
- Are there missing values? 
- What are other issues with the dataset that you might want to fix before training a model?
- How could one fix those issues?

By default, `pandas_profiling` computes many statistics, which can take some time. For this reason, we use the `minimal=True` option, which disables the calculation of correlations and dynamic binning.
You can speed it up even more by subsampling the data first.

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> use <code>df.sample(500)</code> to get a random sample of 500 rows, e.g. <code>df_train_stratified.sample(500)</code>. </li>
    <li> the function call might then look like <code>pandas_profiling.ProfileReport(df_train_stratified.sample(500))</code> </li>
</ul>
</details>

You can use `profile.to_file(output_file="your_report.html")` to save the report to an HTML file, which can be needed if you run this exercise on Google Colab. 



In [ ]:
profile = pandas_profiling.ProfileReport(#fillme, minimal=True)

In [ ]:
profile.to_notebook_iframe()

### 3.2. Correlations

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Plot some features against the target property and calculate the Pearson and Spearman correlation coefficient (what is the different between those?) 
- What are the strongest correlations? 
- *Optional:* Do they change if you switch from CO$_2$ to CH$_4$ uptake as the target instead? Explain your observation.

To get the correlation matrices, you can use the `df.corr(method=)`method on your dataframe (`df`).

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> To get the correlation with a target, you can use indexing. E.g. <code>df.corr(method='spearman')[TARGET]</code></li>
    <li> To sort numpy arrays, use <code>.sort()</code> </li>
      <li> You can use <code>scatter = hv.Scatter(gaussian_mix, 'x', ['y', 'color']).opts(color='color', cmap='rainbow')</code> for plotting </li>
</ul>
</details>

$\color{DarkRed}{\textsf{Tip}}$
 
The atomic structures of the MOFs have been deposited on the [Materials Cloud](https://www.materialscloud.org/discover/mofs#mcloudHeader). 
You can use the identifier from the "MOFname" column to visualize the structure of any material directly from the notebook via the `view_structure` function. For example:

In [ ]:
view_structure('str_m2_o10_o29_pcu_sym.69')

## 4. Baselines

For machine learning, it is important to have some *baselines* to which one then compares the results of a model. 

A baseline could be a really simple model, a basic heuristic or the current state of the art.
this. We will use a heuristic.

For this we use sklearn `Dummy` objects that simply calculate the mean, the median or the most frequent case of the training set, when you run the `fit()` method on them (which takes the features matrix $\mathbf{X}$ and the labels $\mathbf{y}$ as arguments.
I.e., the prediction of a `DummyRegressor` with `mean` strategy will always be the mean, independent of the input. 

Using these objects, instead of the mean directly, allows you to easily swap them with other models in pipelines, where one chains many data transformation steps (see section 6).

### 4.1. Build dummy models

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Create [`DummyRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html) instances for  `mean`, `median`. (e.g. `dummyinstance = DummyRegressor(strategy='')`)
- Train them on the training data (`dummyinstance.fit(df_train[FEATURES], df_train[TARGET_BINARY])`)

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> to create <code>DummyRegressor</code> you can for example use <code> dummyregressor_mean = DummyRegressor(strategy='mean') </code> </li>
</ul>
</details>

In [ ]:
# Build DummyRegressors
dummyregressor_mean = DummyRegressor(strategy='mean')
dummyregressor_median = DummyRegressor(#fillme)

In [ ]:
# Fit Dummy Regressors
dummyregressor_mean.fit(df_train_stratified[FEATURES], df_train_stratified[TARGET])
dummyregressor_median.fit(#fillme)

#### Evaluate the performance of the dummy models

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Calculate maximum error, mean absolute error and mean square error for the dummy regressors on training and test set. What would you expect those numbers to be?
- Do the actual values surprise you? 
- What does this mean in practice for reporting of metrics?

It can be handy to store our metrics of choice in a nested dictionary: 

```python
{'dummyestimator1': {
                        'metric_a_key': metric_a_value, 
                        'metric_b_key': metric_b_value
                    },
 'dummyestimator2': {
                        'metric_a_key': metric_a_value, 
                        'metric_b_key': metric_b_value
                    },
 }
``` 

You will now write functions `get_regression_metrics(model, X, y_true)` that compute the metrics and return the dictionary for a given model. The `predict` method takes the feature matrix $\mathbf{X}$ as input.

In them, we calculate 

$\mathrm {MAE} =\frac{\sum _{i=1}^{n}\left|Y_{i}-\hat{y}_{i}\right|}{n}.$

and 

$\mathrm {MSE} = {\frac {1}{n}}\sum _{i=1}^{n}(Y_{i}-{\hat {Y_{i}}})^{2}.$ 

as well as the maximum error.

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> to perform a prediction using a estimator object, you can call <code> classifier.predict(X) </code> </li>
    <li> to calculate metrics, you can for example call <code>accuracy_score(true_values, predicted_values) </code> </li>
</ul>
</details>

In [ ]:
def get_regression_metrics(model, X, y_true): 
    """
    Get a dicionary with regression metrics:
    
    model: sklearn model with predict method
    X: feature matrix
    y_true: ground truth labels (predicted values)
    """
    y_predicted = model.predict(#fillme)
    
    mae = mean_absolute_error(#fillme)
    mse = mean_squared_error(#fillme)
    maximum_error = max_error(#fillme)
    
    metrics_dict = {
        'mae': mae, 
        'mse': mse, 
        'max_error': maximum_error
    }
    
    return metrics_dict

In [ ]:
dummy_regressors = [
    ('mean', dummyregressor_mean), 
    ('median', dummyregressor_median)
]

In [ ]:
dummy_regressor_results_test = {} # initialize empty dictionary
dummy_regressor_results_train = {}
for regressorname, regressor in dummy_regressors: 
    dummy_regressor_results_test[regressorname] = get_regression_metrics(#fillme)
    dummy_regressor_results_train[regressorname] = get_regression_metrics(#fillme)

## 5. Build actual regression models

Let's build a simple kernel ridge regression (KRR) machine learning model and train it with our raw data.
You can try different kernels, but we recommend to start with the Gaussian radial basis function ('rbf') kernel.
 
 $\color{DarkBlue}{\textsf{Short Question}}$
- Do you expect this model to perform better than the dummy models?
- Train it and then calculate the performance metrics on the training and test set. How do they compare to the performance of the dummy models?

In [ ]:
# Train the model
krr = KernelRidge(kernel='rbf')
krr.fit(#fillme)

In [ ]:
# get the metrics on the train and the test set

## 6. Evaluate the model performance in detail

We have trained our first machine learning model!
We'll first have a closer look at its performance, before learning how to improve it.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Create a parity plot (true values against predictions)for the training and test data
- plot a histogram of the distribution of the training and test errors on the training and test set. Plot the errors also as a function of the true value
- Let's assume we would like to use our model for pre-screening a library of millions of porous materials to zoom-in on those with the most promising gas uptake. Could you tolerate the errors of your model?
- compare the parity plots for this model with the ones for the dummy models. 
Use the plotting functions below the evaluate all the following models you train.

For this exercise, it can be handy to save the results in a dictionary, e.g. 
```(python)
res_train = {
    'y true': ,
    'y pred': 
}
```

As you will need to run this multiple times, it can be useful to make a function that can create such a dictionary.

<details>
<summary> <font color='green'>Click here for hints for plotting</font></summary>
<ul>
    <li> If you want to use matplotlib, you can use the <a href="https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist2d.html">hist2d function</a> </li>
    <li>to create the frequencies and the edges of a histogram, one can use <code>np.histogram</code></li>
</ul>
</details>

In [ ]:
# Create dictionaries with training and test results 
res_train = {
    'y true': # fillme,
    'y pred': # fillme
}

res_test = {
    'y true': # fillme,
    'y pred': # fillme
}

In [ ]:
res_train['error'] = res_train['y true'] - res_train['y pred']
res_test['error'] = res_test['y true'] - res_test['y pred']

# plot it
hv.extension('bokeh')
hex_train = hv.HexTiles(res_train, ['y true', 'y pred']).hist(dimension=['y true','y pred'])
hex_test = hv.HexTiles(res_test, ['y true', 'y pred']).hist(dimension=['y true', 'y pred'])
hex_train + hex_test

## 7. Improve the model

Our data set still has a couple of issues you might have noticed:
- The feature values are not scaled (different features are measured in different units ...)
- Some features are basically constant, i.e. do not contain relevant information and just artificually increase the dimensionality of the problem 
- Some feature distributions are skewed (which is more relevant for some models than for others ...)
- There are only few good materials (also the target distribution is skewed), so if we weight all materials equally we might not do a good job in predicting the best materials

### 7.1. Standard scaling and building a first pipeline 

Given that we will now go beyond training a single model, we will build [Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), which are objects that can collect a selection of transformations and estimators. This makes it quite easy to apply the same set of operations to different datasets.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Build a pipline that first performs standard scaling and then a KRR. Call it `pipe_w_scaling`. 

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> the <code>fit</code>, <code>predict</code> methods also work for pipelines </li>
</ul>
</details>

In [ ]:
pipe_w_scaling = Pipeline(
   [
       ('scaling', StandardScaler()), 
       ('krr', #fillme)
   ]
)

### 7.2. Hyperparameter optimization

The most approach to hyperparameter optimization is to define a grid of all relevant parameters and to search over the grid for the best model performance.

$\color{DarkBlue}{\textsf{Short Exercise}}$
- Think about which parameters you could optimize in the pipeline. Note that your KRR model has parameters you can optimize. You can also switch off some steps by setting them to `None'.
- For each parameter you need to define a resonable grid to search over.
- Run the hyperparameter optimization using 5-fold cross-validation (you can adjust the number of folds according to your computational resources/impatience. It turns out at k=10 is the [best tradeoff between variance and bias](https://arxiv.org/abs/1811.12808)). 
Tune the hyperparameters until you are statisfied (e.g., until you cannot improve the cross validated error any more)
- Why don't we use the test set for hyperparameter tuning? 
- Evaluate the model performance by calculating the performance metrics (MAE, MSE, max error) on the training and the test set.
- Instead of grid search, try to use random search on the same grid (`RandomizedSearchCV`) and fix the number of evaluations (`n_iter`) to a fraction of the number of evaluations of grid search. What do you observe and conclude?

 $\color{DarkRed}{\textsf{Tips}}$
- If you want to see what is happening, set the verbosity argument of the `GridSearchCV` object to a higher number.
 
- If you want to speed up the optimization, you can run it in parallel by setting the `n_jobs` argument to the number of workers. If you set it to -1 it will use all available cores.
 
- If the optimization is too slow, reduce the number of data points in your set, the number of folds or the grid size. Note that it can also be a feasible strategy to first use a coarser grid and the a finer grid for fine-tuning.

- For grid search, you need to define a parameter grid, which is a dictionary of the following form: 
```(python)
param_grid = {
                    'pipelinestage__parameter': np.logspace(-4,1,10),
                    'pipelinestage': [None, TransformerA(), TransformerB()]
            }
```

- After the search, you can access the best model with `.best_estimator_` and the best parameters with `.best_params_` on the GridSearchCV instance

- If you initialize the GridSearchCV instance with `refit=True` it will automatically train the model with all training data (and not only the training folds from cross-validations)

The double underscore (dunder) notation works recursively and specifies the parameters for any pipeline stage. For example, `ovasvm__estimator__cls__C` would specifiy the `C` parameter of the estimator in the one-versus-rest classifier. 

You can print all parameters of the pieline using `pp.pprint(sorted(pipeline.get_params().keys()))`

<details>
<summary> <font color='green'>Click here for hints about pipelines and grid search</font></summary>
<ul>
    <li> You can use the <code>np.logspace</code> function to generate a grid for values that you want to vary on a logarithmic scale </li>
    <li> There are two hyperparameters for KRR: the regularization strength <code>alpha</code> and the Gaussian width  <code>gamma</code> </li>
    <li> For the regularization strength, values between 1 and 1e-3 can be reasonable. For gamma you can use the median heuristic, gamma = 1 / median, or values between 1e-3 and 1e3</li>
</ul>
</details>

In [ ]:
# Define the parameter grid and the grid search object
param_grid = {
                    'scaling': [MinMaxScaler(), StandardScaler()],
                    'krr__alpha': #fillme,
                    'krr__#fillme': #fillme
            }

grid_krr = GridSearchCV(#your pipeline, param_grid=param_grid, 
                        cv=#number of folds, verbose=2, n_jobs=-1)

random_krr = RandomizedSearchCV(#your pipeline, param_distributions=param_grid, n_iter=#number of evaluations,
                        cv=#number of folds, verbose=2, n_jobs=-1)

In [ ]:
# run the grid search by calling the fit method 
grid_krr.fit(#fillme)
random_krr.fit(#fillme)

In [ ]:
# get the performance metrics
get_regression_metrics(#fillme)

<details>
<summary> <font color='green'>Click here for some more information about hyperparameter optimization</font></summary>
Grid search is not the most efficient way to perform hyperparamter optimization. Even <a href="http://jmlr.csail.mit.edu/papers/volume13/bergstra12a/bergstra12a.pdf">random search was shown to be more efficient</a>. Really efficient though are Bayesian optimization approaches like <a href='https://papers.nips.cc/paper/4443-algorithms-for-hyper-parameter-optimization.pdf)'>TPE</a>. This is implemented in the hyperopt library, which is also installed in your conda environment.
</details>

<details>
<summary> <font color='green'>Click here for hyperparameter optimization with hyperopt (advanded and optional outlook)</font></summary>
    
<b>Import the tools we need</b>
<code>
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, mix, rand, anneal, space_eval
from functools import partial
</code>    

<b>Define the grid</b>
<code>
param_hyperopt = {
    "krr__alpha": hp.loguniform("krr__alpha", np.log(0.001), np.log(10)),
    "krr__gamma": hp.loguniform("krr__gamma", np.log(0.001), np.log(10)),
}
</code> 

<b>Define the objective function</b>
<code>
def objective_function(params):
    pipe.set_params(
        **{
            "krr__alpha": params["krr__alpha"],
            "krr__gamma": params["krr__gamma"],
        }
    )
    score = cross_val_score(
        pipe, X_train, y_train, cv=10, scoring="neg_mean_absolute_error"
    ).mean()
    return {"loss": -score, "status": STATUS_OK} 
</code>

<b>We will use a search in which we mix random search, annealing and tpe</b>
<code>
trials = Trials()
mix_search = partial(
   mix.suggest,
   p_suggest=[(0.15, rand.suggest), (0.15, anneal.suggest), (0.7, tpe.suggest)],
)
</code>

<b>Now, we can minimize the objective function.</b>
<code>
best_param = fmin(
        objective_function,
        param_hyperopt,
        algo=mix_search,
        max_evals=MAX_EVALES,
        trials=trials,
        rstate=np.random.RandomState(RANDOM_SEED),
    )
</code>

</details>

## 8. Submit your results to Kaggle (optional)

Join the [Kaggle competition](http://www.kaggle.com/c/molsim2020) for this course! There we deposited some features that you have not seen before. Use your model to predict the CO$_2$ uptake for the structures there. Tune your model to get the best predictions as you move trough this notebook! Also feel free to explore other models.

Create `submission.csv` with your predictions to join the competition and upload it to the competition site.

In [ ]:
kaggle_data = pd.read_csv('data/features.csv')
kaggle_predictions = #fillme.predict(kaggle_data[FEATURES])

In [ ]:
submission = pd.DataFrame({
    "id": kaggle_data["id"],
    "prediction": kaggle_predictions
})

submission.to_csv("submission.csv", index=False)

Once you have created this file, you can had over to the [submission page](https://www.kaggle.com/c/molsim2020/submit) and upload your file.

![Kaggle submission](_static/kaggle_upload.png)

## 9. Feature Engineering 

Finally, we would like to remove features with low variance. This can be done by setting a variance threshold.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Add a variance threshold to the pipeline (select the correct function argument)
- Use random search for hyperparameter optimization, retrain the pipeline, and calculate the performance metrics (max error, MAE, MSE) on the training and test set

In [ ]:
# Define the pipeline
pipe_variance_threshold = Pipeline(
    # fillme with the pipeline steps
    [
        ('variance_treshold', VarianceThreshold(#fillme with threshold)), 
        #fillme with remaining pipeline steps
    ]
)

In [ ]:
param_grid_variance_threshold = {
                    'scaling__parameter': [None, StandardScaler()],
                    'krr__alpha': #fillme,
                    'krr__#fillme': #fillme,
                    'variance_treshold__threshold': #fillme
            }

random_variance_treshold = RandomizedSearchCV(#your pipeline, param_distributions=param_grid, n_iter=#number of evaluations,
                        cv=#number of folds, verbose=2, n_jobs=-1)

In [ ]:
# Fit the pipeline and run the evaluation
random_variance_treshold.fit(#fillme)

$\color{DarkBlue}{\textsf{Short Exercise (optional)}}$
- replace the variance threshold with a model-based feature selection 
`('feature_selection', SelectFromModel(LinearSVC(penalty="l1")))`

## 10. Saving the model

Now, that we spent so much time in optimizing our model, we do not want to loose it. 

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- use the [joblib library](https://scikit-learn.org/stable/modules/model_persistence.html) to save your model
- make sure you can load it again

Use this to save models as you go.

In [ ]:
# Dump your model
joblib.dump(model, filename)

In [ ]:
# Try to load it again
model_loaded = joblib.load(filename)

## 11. Influence of Regularization (optional)

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- what happens if you set $\alpha=0$ or to large value? Why is this the case?

 To test this, fix this value in one of your pipelines, retrain the models (re-optimizing the other hyperparameters) and rerun the performance evaluation.

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> Check the derivation for ridge regression and KRR in the notes. </li>
</ul>
</details>

## 12. Interpreting the model

Now, that our model performs decently, we would like to know which features are mainly responsible for this, i.e. how the model performs its reasoning. 

One method to do so is the [permutation feature importance technique](https://christophm.github.io/interpretable-ml-book/feature-importance.html).

$\color{DarkBlue}{\textsf{Short question}}$

We use both descriptors that encode the pore geometry (density, pore diameters, surface areas) as well as some that describe the chemistry of the MOF (the RACs). 
- Would you expect the relative importance of these features to be different for prediction of gas adsorption at high vs low gas pressure?

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> <a href="https://pubs.acs.org/doi/abs/10.1021/acs.chemmater.8b02257">An article from Diego et al.</a> (10.1021/acs.chemmater.8b02257) gives some hints.</li>
</ul>
</details>

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Complete the function `_calculate_permutation_scores` (which we took from the `sklearn` package) and which is needed to calculate the permutation feature importance using the `permutation_importance` function. 

In [ ]:
def _calculate_permutation_scores(estimator, X, y, col_idx, random_state,
                                  n_repeats, scorer):
    """Calculate score when `col_idx` is permuted. Based on the sklearn implementation
    
    estimator: sklearn estimator object
    X: pd.Dataframe or np.array
    y: pd.Dataframe or np.array
    col_idx: int
    random_state: int
    n_repeats: int 
    scorer: function that takes model, X and y_true as arguments
    """
    random_state = check_random_state(random_state)

    X_permuted = X.copy()
    scores = np.zeros(n_repeats)
    # get the indices
    shuffling_idx = np.arange(X.shape[0])
    for n_round in range(n_repeats):
        # shuffle them (fill in what you want to shuffle)
        random_state.shuffle(#fillme)  
        
        # Deal with dataframes
        if hasattr(X_permuted, "iloc"):
            # .iloc selects the indices from a dataframe and yougive it [row, column]
            col = X_permuted.iloc[#fillme]
            col.index = X_permuted.index
            X_permuted.iloc[:, col_idx] = col
            
        # Deal with numpy arrays 
        else:
            # array indexing is [row, column]
            X_permuted[:, col_idx] = X_permuted[#fillme]
        
        # Get the scores
        feature_score = scorer(estimator, X_permuted, y)
        
        # record the scores in array 
        scores[n_round] = feature_score

    return scores

In [ ]:
def permutation_importance(estimator, X, y, scoring='neg_mean_absolute_error', n_repeats=5,
                           n_jobs=-1, random_state=None):
    """Permutation importance for feature evaluation 
    estimator : object
        An estimator that has already been :term:`fitted` and is compatible
        with :term:`scorer`.
    X : ndarray or DataFrame, shape (n_samples, n_features)
        Data on which permutation importance will be computed.
    y : array-like or None, shape (n_samples, ) or (n_samples, n_classes)
        Targets for supervised or `None` for unsupervised.
    scoring : string, callable or None, default=None
        Scorer to use. It can be a single
        string (see :ref:`scoring_parameter`) or a callable (see
        :ref:`scoring`). If None, the estimator's default scorer is used.
    n_repeats : int, default=5
        Number of times to permute a feature.
    n_jobs : int or None, default=None
        The number of jobs to use for the computation.
        `None` means 1 unless in a :obj:`joblib.parallel_backend` context.
        `-1` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.
    random_state : int, RandomState instance, or None, default=None
        Pseudo-random number generator to control the permutations of each
        feature. See :term:`random_state`.
    """
    # Deal with dataframes
    if not hasattr(X, "iloc"):
        X = check_array(X, force_all_finite='allow-nan', dtype=None)

    # Precompute random seed from the random state to be used
    # to get a fresh independent RandomState instance for each
    # parallel call to _calculate_permutation_scores, irrespective of
    # the fact that variables are shared or not depending on the active
    # joblib backend (sequential, thread-based or process-based).
    random_state = check_random_state(random_state)
    random_seed = random_state.randint(np.iinfo(np.int32).max + 1)
     
    # Determine scorer from user options.
    scorer = check_scoring(estimator, scoring=scoring)
    # get the performance score on the unpermuted data 
    baseline_score = scorer(estimator, X, y)
    
    # run the permuted evaluations in parallel for each column
    scores = Parallel(n_jobs=n_jobs)(delayed(_calculate_permutation_scores)(
        estimator, X, y, col_idx, random_seed, n_repeats, scorer
    ) for col_idx in range(X.shape[1]))
    
    # get difference two
    importances = baseline_score - np.array(scores)
    
    # return the results (dictionary)
    return Bunch(importances_mean=np.mean(importances, axis=1),
                 importances_std=np.std(importances, axis=1),
                 importances=importances)

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Use your function to find the five most important features.
- Which are they? Did you expect this result?

In [ ]:
permutation_results = permutation_importance(#fillme)

In [ ]:
permutation_results['features'] = FEATURES
bars = hv.Bars(permutation_results, 'features', ['importances_mean', 'importances_std']).sort('importances_mean', reverse=True)
errors = hv.ErrorBars(permutation_results, 'features', vdims=['importances_mean', 'importances_std']).sort('importances_mean', reverse=True)

bars * errors

<details>
<summary> <font color='green'>Click here for hints</font></summary>
<ul>
    <li> To get the top <emph>n</emph> indices of an array <code>a</code>, you can use <code>np.argsort(a)[-n:]</code></li>
    <li> Get the feature names from the <code>FEATURES</code> list </li> 
</ul>
</details>

<details>
<summary> <font color='green'>Click here for more information on model interpretation</font></summary>
The permutation feature importance technique is not a silver bullet, e.g. there are issues with correlated features.
However, it is likely <a href='https://explained.ai/rf-importance/'>a better choice than feature importance, like impurity decrease, derived from random forests</a>).
</details>

## 13. PCA (optional) 

As you may have noticed, our data is high-dimensional (see `df_train.shape`). 
It's hard to visualize such data.

$\color{DarkBlue}{\textsf{Short Exercise}}$
- How many features did we use?

One way to visualize high dimensional data is to use [principal component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) which helps you find those linear combinations of features that explain most of the variance in the data. 
But this method also has some caveats, which we will explore later in this exercise, and which non-linear techniques like [t-sne](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) try to avoid. 

One can then plot the data in two dimensions in terms of the first two principal components.

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- plot the explained variance as a function of the number of principal components and plot a [scree plot](https://en.wikipedia.org/wiki/Scree_plot). Use all features for this analysis (on the training set). Would it be possible to compress this dataset using PCA? 
- plot the dataset in terms of the first two principal components. What are the features that are most important in those first principal components (look at the loadings of the first two principal components). 

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> A scree plot shows the explained variance as a function of the number of components. For this, we run <code>PCA(n_components=n_features)</code>, where <code>n_components=X.shape[1] </code> </li>
    <li> The <code> PCA </code> object also has a <code> fit </code> method </li>
</ul>
</details>

In [ ]:
# Build the PCA object and run it 
pca = PCA(n_components=#fillme)
pca.fit(StandardScaler().fit_transform(df_train_stratified[#fillme]))

In [ ]:
# Plot the scree plot
data = {
            'feature' : np.arange(0, len(#fillme)),
            'explained variance ratio': pca.explained_variance_ratio_
        }

hv.Curve(data, 'feature', 'explained variance ratio')

#### PCA on Swiss Roll  

Some data comes in interesting shapes. 
The Swiss roll dataset was created to test out 
dimensionality reduction algorithms by creating some data in 2D and mapping it then to 3D  using a smooth function (here: $(x,y,z) := (x \cos (x), y, x \sin(x))$).

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- Create some Swiss roll data - first use the `gaussian_mixture_data` to create Gaussian mixture data in 2D, then map it to 3D using `map_to_3d`. 
- Visualize the data first in 2D and, after the mapping, in 3D. 

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> Both <code>means</code> and <code>covariances</code> take lists as input.  </li>
    <li> The list for the means contains tuples of (x,y) coordinates for the centers of the Gaussian blobs and the list for the covariances takes floats. </li>
    <li> try e.g. <code> means = [(-2, -2), (-2,2), (2,-2), (2,2)], covariances = [1,1,1,1] </code> </li>
    <li> for plotting in matplotlib you can use the <a href="https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.scatter.html"><code>plt.scatter</code></a>. You can use the <code>c</code> argument to color the points.
</ul>
</details>

In [ ]:
def gaussian_mixture_data(means: list=[(-2, -2), (1,1)], 
                          covariances: list=[1], numpoints: int=200) -> pd.DataFrame:
    """
    generate some gaussian mixture data
    """
    from sklearn.datasets import make_gaussian_quantiles
    assert len(means) == len(covariances)
    
    X = []
    Y = []
    c = []
    
    # loop over all means and covariances
    for i, mucov in enumerate(zip(means, covariances)):
        mu, cov = mucov
        print(f'making blob of mean {mu} and covariance {cov}')
        x, y = make_gaussian_quantiles(mean=mu, cov=cov,
                                         n_samples=numpoints, 
                                         random_state=RANDOM_SEED)
        X.append(x[:,0])
        Y.append(x[:,1])
        c.append([i  + 1] * numpoints)
    
    gaussian_mix = {
        'x': np.array(X).flatten(),
        'y': np.array(Y).flatten(),
        'color': np.array(c).flatten() 
    }
    
    return pd.DataFrame.from_dict(gaussian_mix)

In [ ]:
gaussian_mix = gaussian_mixture_data(##fillme)

##### Plot it 

In [ ]:
# plot it
scatter = hv.Scatter(gaussian_mix, 'x', ['y', 'color']).opts(color='color', cmap='rainbow')
scatter

##### Map it to 3D and plot it

In [ ]:
def map_to_3d(x: float, y:float) -> list: 
    """
    x cos (x), y, x sin(x)
    Feel free to play aroung with it
    """
    return [.5 * x * np.cos(x), y,   2 * x * np.sin(x)]

$\color{DarkBlue}{\textsf{Short Exercise}}$
- Replace `#fillme` by the correct function arguments in the list comprehension.
- Plot the result in three dimensions using the 'color' column for coloring

In [ ]:
coordinates = np.array([map_to_3d(#fillme) for x,y in zip(gaussian_mix['x'], gaussian_mix['y'])])

gaussian_mix['x'] = coordinates[:,0]
gaussian_mix['y'] = coordinates[:,1]
gaussian_mix['z'] = coordinates[:,2]

In [ ]:
# Plot the data in 3d
scatter3d = hv.Scatter3D(#fillme, kdims=['x', 'y'], vdims=['z', 'color'])
scatter3d = scatter3d.opts(
            opts.Scatter3D(color='color',  cmap='rainbow'))
scatter3d

##### Run the PCA transformation

Now, let's project the data from feature space onto its first two principal components, and then plot the data in two dimension in these coordinates.

$\color{DarkBlue}{\textsf{Short Exercise}}$
- Project the three dimensional swiss roll data onto the first two principal components
- Plot the data in two dimensions, use the original coloring (which could e.g. represent different material classes)


<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> To perform the the transformation, you can use the <code>fit_transform()</code> method of the <code>PCA(n_components=2)</code> instance</li>
</ul>
</details>



In [ ]:
# run the PCa
pca = PCA(#fillme)
swissroll_transformed = pca.fit_transform(#fillme)

In [ ]:
# plot it
swissroll_transformed_dict = {}
swissroll_transformed_dict['x'] = swissroll_transformed[:,#fill with a integer]
swissroll_transformed_dict['y'] = swissroll_transformed[:,#fill with a integer]
swissroll_transformed_dict['color'] = gaussian_mix['color']
scatter = hv.Scatter(swissroll_transformed_dict, 'x', ['y', 'color']).opts(
        color='color', cmap='rainbow')
scatter

 $\color{DarkBlue}{\textsf{Short Exercise}}$
- If you look at the 2D-plot, what would you have expected a dimensionality reduction to do? What did actually happen? 
- What happens to the transformation if you scale one feature? What does it mean in practice when you e.g. want to understand the importance of features in terms of their variance?

<details>
<summary> <font color='green'>Click here for a hint</font></summary>
<ul>
    <li> To standardize one variable you can use <code>gaussian_mix['x'] = (gaussian_mix['x']- gaussian_mix['x'].mean())/gaussian_mix['x'].std()</code> - Or just try to multiply by a constant </li>
    <li> For serious applications, always look at the data in the dataframe first... the same quantity might come in different units! </li>
</ul>
</details>


## Additional research questions (optional)

- Change the target from CO$_2$ to methane. Is it easier to predict? What changes in the feature importance?
- Do also a search over model space. Do you find better performance with a Gradient Boost model? 